# [SQLite as a Vector Store with SQLiteVec](https://python.langchain.com/docs/integrations/vectorstores/sqlitevec/)

This notebook covers how to get started with the SQLiteVec vector store.

```
**SQLite-Vec** is an SQLite extension designed for vector search, emphasizing local-first operations and easy integration into applications without external servers. It is the successor to **SQLite-VSS** by the same author. It is written in zero-dependency C and designed to be easy to build and use.
```

This notebook shows how to use the SQLiteVec vector database.

In [ ]:
# You need to install sqlite-vec as a dependency.
%pip install --upgrade --quiet  sqlite-vec

In [43]:
import sqlite3
import sqlite_vec
from langchain_community.vectorstores.sqlitevec import SQLiteVec
from langchain.embeddings import SentenceTransformerEmbeddings

# Initialize your embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Create a connection to your SQLite database
db_path = "./tmp/vec.db"
con = sqlite3.connect(db_path)
con.enable_load_extension(True)

# sqlite_vec.load(con)

# Load the sqlite-vec extension
# con.execute("SELECT load_extension('sqlite_vec')")

# Initialize the SQLiteVec instance
vector_store = SQLiteVec(table="state_union", connection=con, embedding=embedding_function)

OperationalError: no such module: vec0

In [44]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import SQLiteVec
from langchain_text_splitters import CharacterTextSplitter

# load the document and split it into chunks
loader = TextLoader("../../../text_files/state_of_the_union.txt")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
texts = [doc.page_content for doc in docs]


# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
connection = SQLiteVec.create_connection(db_file="./tmp/vec.db")

db1 = SQLiteVec(
    table="state_union", embedding=embedding_function, connection=connection
)

Created a chunk of size 304, which is longer than the specified 300
Created a chunk of size 332, which is longer than the specified 300
Created a chunk of size 325, which is longer than the specified 300


In [45]:
for text in texts:
    db1.add_texts([text])

In [ ]:
db1.add_texts(["Ketanji Brown Jackson is awesome"])

In [46]:
# query it again
query = "What did the president say about Ketanji Brown Jackson"
data = db1.similarity_search(query)

# print results
for row in data:
    print(row.page_content)
    print("=======================")

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
We don’t know for sure if a burn pit was the cause of his brain cancer, or the diseases of so many of our troops. 

But I’m committed to finding out everything we can. 

Committed to military families like Danielle Robinson from Ohio. 

The widow of Sergeant First Class Heath Robinson.
He will never extinguish their love of freedom. He will never weaken the resolve of the free world. 

We meet tonight in an America that has lived through two of the hardest years this nation has ever faced. 

The pandemic has been punishing.
As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential.


In [41]:
# query it again
query = "What did the president say about Today?"
data = db1.similarity_search(query)

# print results
for row in data:
    print(row.page_content)
    print("=======================")

And my report is this: the State of the Union is strong—because you, the American people, are strong. 

We are stronger today than we were a year ago. 

And we will be stronger a year from now than we are today. 

Now is our moment to meet and overcome the challenges of our time.
Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again.
But that trickle-down theory led to weaker economic growth, lower wages, bigger deficits, and the widest gap between those at the top and everyone else in nearly a century. 

Vice President Harris and I ran for office with a new economic vision for America.
I remember when my Dad had to leave our home in Scranton, Pennsylvania to find work. I grew up in a family where if the price of food went up, you felt it. 

That’s why one of the first things I did as President was figh